In [2]:

from re import sub
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
import numpy as np


def semantic_sim(query_string) :
    class_nature = ['A boy saw a frog skipping and jumping into pond', 'When she was walking in a forest, she saw an animal', 'They watched the sun set and the birds chip from the hill', 'Trees are useful gifts of nature as they give us flowers, timber, bamboo and fuels', 'Nature consists of animal kingdom, plant kingdom and mineral kingdom']
    class_wildlife = ['We know that tigers are highly adaptable and have been documented at various altitudes in different landscapes', 'Carnivorous animals like lions, tigers, bears and leopards always flock in dense forests', 'Sloths are slow-moving creatures that spend their lives in tree canopies, munching on leaves and napping', 'African elephants in Namibia, black rhinos, giant pandas, and mountain gorillas are just some of the species whose populations have recovered in recent years', 'Tools like collars powered by GPS tracking are helping monitor wildlife and reduce human-wildlife conflict']

    stopwords = ['the', 'and', 'are', 'a', 'at', 'in', 'as', 'on', 'into']

    def preprocess(doc):
        # Tokenize, clean up input document string
        #doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
        #doc = sub(r'<[^<>]+(>|$)', " ", doc)
        #doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
        #doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
        return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

    # Preprocess the documents, including the query string
    corpus_nature = [preprocess(it) for it in class_nature]
    corpus_wildlife = [preprocess(it) for it in class_wildlife]
    query = preprocess(query_string)

    # Load the model: this is a big file, can take a while to download and open
    glove = api.load("glove-wiki-gigaword-50")    
    similarity_index = WordEmbeddingSimilarityIndex(glove)

    # Build the term dictionary, TF-idf model
    dictionary_nature = Dictionary(corpus_nature+[query])
    tfidf_nature = TfidfModel(dictionary=dictionary_nature)

    dictionary_wildlife = Dictionary(corpus_wildlife+[query])
    tfidf_wildlife = TfidfModel(dictionary=dictionary_wildlife)

    # Create the term similarity matrix.  
    similarity_matrix_nature = SparseTermSimilarityMatrix(similarity_index, dictionary_nature, tfidf_nature)
    similarity_matrix_wildlife = SparseTermSimilarityMatrix(similarity_index, dictionary_wildlife, tfidf_wildlife)


    # Compute Soft Cosine Measure between the query and the documents.
    # From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb

    query_tf_nature = tfidf_nature[dictionary_nature.doc2bow(query)]
    query_tf_wildlife = tfidf_wildlife[dictionary_wildlife.doc2bow(query)]

    index_nature = SoftCosineSimilarity(
                tfidf_nature[[dictionary_nature.doc2bow(document) for document in corpus_nature]],
                similarity_matrix_nature)
    index_wildlife = SoftCosineSimilarity(
                tfidf_wildlife[[dictionary_wildlife.doc2bow(document) for document in corpus_wildlife]],
                similarity_matrix_wildlife)

    doc_similarity_scores_nature = index_nature[query_tf_nature]
    doc_similarity_scores_wildlife = index_wildlife[query_tf_wildlife]

    # Output the sorted similarity scores and documents
    avg_nature=0
    #counter_nature=0
    avg_wildlife=0
    #counter_nature=0

    sorted_indexes_nature = np.argsort(doc_similarity_scores_nature)[::-1]
    for idx in sorted_indexes_nature:
        #print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')
        #counter_nature+=1
        avg_nature+=doc_similarity_scores_nature[idx]
    #avg_nature/=counter_nature
    avg_nature/=5.0

    sorted_indexes_wildlife = np.argsort(doc_similarity_scores_wildlife)[::-1]
    for idx in sorted_indexes_wildlife:
        #print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')
        #counter_wildlife+=1
        avg_wildlife+=doc_similarity_scores_wildlife[idx]
    #avg_nature/=counter_nature
    avg_wildlife/=5.0

    if avg_nature<0.3 and avg_wildlife<0.3:
        return 3
    elif avg_nature>avg_wildlife:
        return 1
    else :
        return 2
# 1    0.688    tomatoes are actually fruit
# 0    0.000    cars drive on the road

###########################ENTER THE QUERY STRING BELOW###########################################
print(semantic_sim(#query string))


3


/home/venkatesh/Downloads/enter/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/home/venkatesh/Downloads/enter/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/home/venkatesh/Downloads/enter/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/home/venkatesh/Downloads/enter/lib/python3.7/site-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
